In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv
!pip install modin[ray]  # Ray 엔진 사용 시
!pip install chromadb
!pip install langchain
!pip install langchain_community
!pip install langchain-huggingface
!pip install sentence-transformers
!pip install -U bitsandbytes transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.8 MB/s eta 0:00:0

# Ver_1

In [ ]:
import os
import re   # 정규표현식 활용
from dotenv import load_dotenv
import pandas as pd   # 데이터 처리
import chromadb
from chromadb import Client
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.schema import Document

# CSV 파일 불러오기
NHTI_df = pd.read_csv('/content/drive/MyDrive/NH 데이콘/본선/slopes_about_log_standardize_with_nhti.csv')

# 임베딩 모델 초기화
emb_model = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# Chroma DB에 CSV 파일을 문서 형식으로 저장
import tqdm
ids = []
metadatas = []
embeddings = []
documents = []

feature_columns = [col for col in NHTI_df.columns if col not in ['tck_iem_cd', 'bse_dt']]

for row in tqdm.tqdm(NHTI_df[:20000].iterrows()):
    index = row[0]
    bse_dt = row[1].bse_dt
    tck_iem_cd = row[1].tck_iem_cd

    metadata = {
        "date": bse_dt,
        "stock": tck_iem_cd
    }

    # feature 값들을 메타데이터에 추가
    for feature in feature_columns:
        metadata[feature] = str(row[1][feature])

    # 임베딩을 위한 쿼리 문자열 생성
    query = f"Stock: {row[1].tck_iem_cd} Date: {row[1].bse_dt} " + " ".join([f"{feature}: {row[1][feature]}" for feature in feature_columns])

    # Document 객체 생성
    doc = Document(page_content=query, metadata=metadata)

    # Document 객체 생성
    doc = Document(page_content=query, metadata=metadata)
    documents.append(doc)

# Chroma DB에 저장
DB_PATH = "/content/chroma_db"
docsearch = Chroma.from_documents(documents, emb_model,
                                  persist_directory=DB_PATH,
                                  collection_name="stocks")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 110812/110812 [00:56<00:00, 1957.96it/s]


KeyboardInterrupt: 

In [ ]:
def calculate_nhti_with_input():
    user_inputs = []

    # 사용자로부터 매수 종목, 매수 수량, 매수 일자 입력 받기
    while True:
        stock_code = input("매수 종목을 입력하세요 (종료하려면 'q' 입력): ")
        if stock_code.lower() == 'q':
            break
        quantity = int(input("매수 수량을 입력하세요: "))
        purchase_date = input("매수 일자를 입력하세요 (YYYY-MM-DD 형식): ")

        user_inputs.append({"매수 종목": stock_code, "매수 수량": quantity, "매수 일자": purchase_date})

    # Step 1: 총 매수 수량 계산
    total_quantity = sum(input_data["매수 수량"] for input_data in user_inputs)

    # Step 2: 가중합 점수 초기화
    weighted_scores = {
        "Risk_Score": 0,
        "Crowd_Psychology_Score": 0,
        "Interest_Score": 0,
        "Volatility_Score": 0
    }

    # Step 3: 각 매수 내역에 대해 데이터 필터링 및 가중치 적용
    for input_data in user_inputs:
        stock_code = input_data["매수 종목"]
        quantity = input_data["매수 수량"]
        purchase_date = input_data["매수 일자"].replace("-", "")

        filtered_data = NHTI_df[(NHTI_df['tck_iem_cd'] == stock_code) & (NHTI_df['bse_dt'] == int(purchase_date))]

        if not filtered_data.empty:
            weight = (quantity / total_quantity) * 100

            weighted_scores["Risk_Score"] += filtered_data.iloc[0]["Risk_Score"] * (weight / 100)
            weighted_scores["Crowd_Psychology_Score"] += filtered_data.iloc[0]["Crowd_Psychology_Score"] * (weight / 100)
            weighted_scores["Interest_Score"] += filtered_data.iloc[0]["Interest_Score"] * (weight / 100)
            weighted_scores["Volatility_Score"] += filtered_data.iloc[0]["Volatility_Score"] * (weight / 100)

    # NHTI 유형 결정 함수
    def determine_nhti_type(risk, crowd, interest, volatility):
        risk_type = 'A' if risk >= 50 else 'B'
        crowd_type = 'G' if crowd >= 50 else 'I'
        interest_type = 'P' if interest >= 50 else 'U'
        volatility_type = 'X' if volatility >= 50 else 'S'
        return f"{risk_type}{crowd_type}{interest_type}{volatility_type}"

    nhti_type = determine_nhti_type(
        weighted_scores["Risk_Score"],
        weighted_scores["Crowd_Psychology_Score"],
        weighted_scores["Interest_Score"],
        weighted_scores["Volatility_Score"]
    )

    # 상위 5개 종목 추천
    NHTI_df['Distance'] = np.sqrt(
        (NHTI_df['Risk_Score'] - weighted_scores['Risk_Score'])**2 +
        (NHTI_df['Crowd_Psychology_Score'] - weighted_scores['Crowd_Psychology_Score'])**2 +
        (NHTI_df['Interest_Score'] - weighted_scores['Interest_Score'])**2 +
        (NHTI_df['Volatility_Score'] - weighted_scores['Volatility_Score'])**2
    )

    top_5_recommendations = NHTI_df.nsmallest(5, 'Distance')
    return nhti_type, weighted_scores, top_5_recommendations


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
import torch

# 생성형 언어 모델 초기화
model_name = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# BitsAndBytesConfig 설정을 통한 8bit 양자화 적용
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # 8bit 모드로 로드
    device_map="auto"                # GPU와 CPU를 자동으로 할당
)

# 모델을 pipeline으로 래핑
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=1500, max_new_tokens=300, truncation=True)

# LangChain의 HuggingFacePipeline을 이용해 LLM 래핑
llm = HuggingFacePipeline(pipeline=pipe)

# 프롬프트 템플릿 정의
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="""
=== NHTI 성향 분석 및 추천 종목 설명 ===

    당신은 사용자의 투자 정보와 제공된 개별 주식 종목 정보(context)를 바탕으로 사용자의 투자 성향을 판별하는 전문가입니다.
    투자 성향 판별 기준 가이드에 따라 사용자의 투자 성향을 분류하여 설명하세요. 출력에서는 가이드의 내용을 포함시키지 마세요.

    사용자 투자 정보:

    Risk Score: 79.60, Crowd Psychology Score: 29.60,
    Interest Score: 30.40, Volatility Score: 0.00,
    NHTI 유형: AIUS


    context(주식 종목 정보):
    - 종목: SOXL, 날짜: 20240802, Risk Score: 71, Crowd Psychology Score: 15, Interest Score: 29, Volatility Score: 5, 거리: 17.72
- 종목: SOXX, 날짜: 20240805, Risk Score: 75, Crowd Psychology Score: 14, Interest Score: 21, Volatility Score: 0, 거리: 18.79
- 종목: SOXL, 날짜: 20240805, Risk Score: 71, Crowd Psychology Score: 21, Interest Score: 45, Volatility Score: 6, 거리: 19.93
- 종목: POWL, 날짜: 20240731, Risk Score: 85, Crowd Psychology Score: 28, Interest Score: 11, Volatility Score: 0, 거리: 20.20
- 종목: TELL, 날짜: 20240722, Risk Score: 86, Crowd Psychology Score: 16, Interest Score: 17, Volatility Score: 2, 거리: 20.24

    결과는 다음 형식을 따르세요:

    - Risk: 성향 (비율) | 성향 (비율)
      설명...

    - Crowd Psychology: 성향 (비율) | 성향 (비율)
      설명...

    - Interest: 성향 (비율) | 성향 (비율)
      설명...

    - Volatility: 성향 (비율) | 성향 (비율)
      설명...

    최종 NHTI 결과:
    NHTI 유형: 성향 코드 (예: BGPS)
    설명...
    """
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
import numpy as np

# CSV 파일 불러오기
NHTI_df = pd.read_csv('/content/drive/MyDrive/NH 데이콘/본선/slopes_about_log_standardize_with_nhti.csv')

def calculate_nhti_with_input():
    user_inputs = []

    # 사용자로부터 매수 종목, 매수 수량, 매수 일자 입력 받기
    while True:
        stock_code = input("매수 종목을 입력하세요 (종료하려면 'q' 입력): ")
        if stock_code.lower() == 'q':
            break
        quantity = int(input("매수 수량을 입력하세요: "))
        purchase_date = input("매수 일자를 입력하세요 (YYYY-MM-DD 형식): ")

        user_inputs.append({"매수 종목": stock_code, "매수 수량": quantity, "매수 일자": purchase_date})

    # 총 매수 수량 계산
    total_quantity = sum(input_data["매수 수량"] for input_data in user_inputs)

    # 가중합 점수 초기화
    weighted_scores = {
        "Risk_Score": 0,
        "Crowd_Psychology_Score": 0,
        "Interest_Score": 0,
        "Volatility_Score": 0
    }

    # 데이터 필터링 및 가중치 적용
    for input_data in user_inputs:
        stock_code = input_data["매수 종목"]
        quantity = input_data["매수 수량"]
        purchase_date = input_data["매수 일자"].replace("-", "")  # YYYYMMDD 형식으로 변환

        # 필터링: 매수 종목과 매수 일자를 기준으로 데이터 필터링
        filtered_data = NHTI_df[(NHTI_df['tck_iem_cd'] == stock_code) & (NHTI_df['bse_dt'] == int(purchase_date))]

        if not filtered_data.empty:
            # 매수 수량을 기준으로 가중치 계산 (100 기준)
            weight = (quantity / total_quantity) * 100

            # 각 점수에 가중치를 적용하여 가중합 점수 계산
            weighted_scores["Risk_Score"] += filtered_data.iloc[0]["Risk_Score"] * (weight / 100)
            weighted_scores["Crowd_Psychology_Score"] += filtered_data.iloc[0]["Crowd_Psychology_Score"] * (weight / 100)
            weighted_scores["Interest_Score"] += filtered_data.iloc[0]["Interest_Score"] * (weight / 100)
            weighted_scores["Volatility_Score"] += filtered_data.iloc[0]["Volatility_Score"] * (weight / 100)

    # 최종 NHTI 타입 결정
    def determine_nhti_type(risk, crowd, interest, volatility):
        risk_type = 'A' if risk >= 50 else 'B'
        crowd_type = 'G' if crowd >= 50 else 'I'
        interest_type = 'P' if interest >= 50 else 'U'
        volatility_type = 'X' if volatility >= 50 else 'S'
        return f"{risk_type}{crowd_type}{interest_type}{volatility_type}"

    nhti_type = determine_nhti_type(
        weighted_scores["Risk_Score"],
        weighted_scores["Crowd_Psychology_Score"],
        weighted_scores["Interest_Score"],
        weighted_scores["Volatility_Score"]
    )

    # 추천 종목 추출
    NHTI_df['Distance'] = np.sqrt(
        (NHTI_df['Risk_Score'] - weighted_scores['Risk_Score'])**2 +
        (NHTI_df['Crowd_Psychology_Score'] - weighted_scores['Crowd_Psychology_Score'])**2 +
        (NHTI_df['Interest_Score'] - weighted_scores['Interest_Score'])**2 +
        (NHTI_df['Volatility_Score'] - weighted_scores['Volatility_Score'])**2
    )

    # 거리 기준 상위 5개 종목 추천
    top_5_recommendations = NHTI_df.nsmallest(5, 'Distance')

    return nhti_type, weighted_scores, top_5_recommendations


In [ ]:
def generate_nhti_analysis(nhti_type, weighted_scores, top_5_recommendations):
    question = f"""
    Risk Score: {weighted_scores['Risk_Score']:.2f}, Crowd Psychology Score: {weighted_scores['Crowd_Psychology_Score']:.2f},
    Interest Score: {weighted_scores['Interest_Score']:.2f}, Volatility Score: {weighted_scores['Volatility_Score']:.2f},
    NHTI 유형: {nhti_type}
    """

    context = "\n".join([
        f"- 종목: {row['tck_iem_cd']}, 날짜: {row['bse_dt']}, Risk Score: {row['Risk_Score']}, "
        f"Crowd Psychology Score: {row['Crowd_Psychology_Score']}, Interest Score: {row['Interest_Score']}, "
        f"Volatility Score: {row['Volatility_Score']}, 거리: {row['Distance']:.2f} "
        f"(이 종목이 사용자의 성향과 맞는 이유를 설명)"
        for _, row in top_5_recommendations.iterrows()
    ])

    # 프롬프트 생성 및 응답 생성
    prompt = prompt_template.format(question=question, context=context)
    response = llm(prompt)

    print("\n=== NHTI 성향 분석 및 추천 종목 설명 ===")
    print(response)

In [ ]:
2# 사용자 입력 기반 NHTI 계산 및 추천 종목 추출
nhti_type, weighted_scores, top_5_recommendations = calculate_nhti_with_input()

# 생성형 AI를 통한 NHTI 성향 분석 및 추천 종목 설명 출력
generate_nhti_analysis(nhti_type, weighted_scores, top_5_recommendations)


매수 종목을 입력하세요 (종료하려면 'q' 입력): A
매수 수량을 입력하세요: 30
매수 일자를 입력하세요 (YYYY-MM-DD 형식): 2024-05-28
매수 종목을 입력하세요 (종료하려면 'q' 입력): A
매수 수량을 입력하세요: 50
매수 일자를 입력하세요 (YYYY-MM-DD 형식): 2024-05-31
매수 종목을 입력하세요 (종료하려면 'q' 입력): A
매수 수량을 입력하세요: 20
매수 일자를 입력하세요 (YYYY-MM-DD 형식): 2024-06-03
매수 종목을 입력하세요 (종료하려면 'q' 입력): q


Both `max_new_tokens` (=300) and `max_length`(=1500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== NHTI 성향 분석 및 추천 종목 설명 ===

=== NHTI 성향 분석 및 추천 종목 설명 ===

    당신은 사용자의 투자 정보와 제공된 개별 주식 종목 정보(context)를 바탕으로 사용자의 투자 성향을 판별하는 전문가입니다.
    투자 성향 판별 기준 가이드에 따라 사용자의 투자 성향을 분류하여 설명하세요. 출력에서는 가이드의 내용을 포함시키지 마세요.

    사용자 투자 정보:
    
    Risk Score: 79.60, Crowd Psychology Score: 29.60, 
    Interest Score: 30.40, Volatility Score: 0.00, 
    NHTI 유형: AIUS
    

    context(주식 종목 정보):
    - 종목: SOXL, 날짜: 20240802, Risk Score: 71, Crowd Psychology Score: 15, Interest Score: 29, Volatility Score: 5, 거리: 17.72
- 종목: SOXX, 날짜: 20240805, Risk Score: 75, Crowd Psychology Score: 14, Interest Score: 21, Volatility Score: 0, 거리: 18.79
- 종목: SOXL, 날짜: 20240805, Risk Score: 71, Crowd Psychology Score: 21, Interest Score: 45, Volatility Score: 6, 거리: 19.93
- 종목: POWL, 날짜: 20240731, Risk Score: 85, Crowd Psychology Score: 28, Interest Score: 11, Volatility Score: 0, 거리: 20.20
- 종목: TELL, 날짜: 20240722, Risk Score: 86, Crowd Psychology Score: 16, Interest Score: 17, Volatility Score: 2, 거리: 20.24

  

# Ver_2

## 파트 1

In [5]:
import os
import re
from dotenv import load_dotenv
import pandas as pd
import chromadb
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
import tqdm

# CSV 파일 불러오기
NHTI_df = pd.read_csv('/content/drive/MyDrive/NH 데이콘/본선/slopes_about_log_standardize_with_nhti.csv')

# 임베딩 모델 초기화
emb_model = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# Chroma DB에 문서 형식으로 데이터 저장
documents = []
feature_columns = [col for col in NHTI_df.columns if col not in ['tck_iem_cd', 'bse_dt']]

for row in tqdm.tqdm(NHTI_df[:20000].itertuples()):
    metadata = {
        "날짜": row.bse_dt,
        "종목": row.tck_iem_cd,
        **{feature: str(getattr(row, feature)) for feature in feature_columns}
    }
    query = f"종목: {row.tck_iem_cd} 날짜: {row.bse_dt} " + " ".join([f"{feature}: {getattr(row, feature)}" for feature in feature_columns])
    doc = Document(page_content=query, metadata=metadata)
    documents.append(doc)

# Chroma DB 설정 및 데이터 저장
DB_PATH = "/content/chroma_db"
docsearch = Chroma.from_documents(documents, emb_model,
                                  persist_directory=DB_PATH,
                                  collection_name="stocks")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

20000it [00:02, 8566.00it/s]


## 파트 2

In [6]:
import numpy as np

def calculate_nhti_with_input():
    user_inputs = []

    # 사용자로부터 매수 종목, 매수 수량, 매수 일자 입력 받기
    while True:
        stock_code = input("매수 종목을 입력하세요 (종료하려면 'q' 입력): ")
        if stock_code.lower() == 'q':
            break
        try:
            quantity = int(input("매수 수량을 입력하세요: "))
            purchase_date = input("매수 일자를 입력하세요 (YYYY-MM-DD 형식): ")
            user_inputs.append({"매수 종목": stock_code, "매수 수량": quantity, "매수 일자": purchase_date})
        except ValueError:
            print("잘못된 입력입니다. 다시 시도하세요.")

    total_quantity = sum(input_data["매수 수량"] for input_data in user_inputs)
    weighted_scores = {key: 0 for key in ["Risk_Score", "Crowd_Psychology_Score", "Interest_Score", "Volatility_Score"]}

    for input_data in user_inputs:
        stock_code = input_data["매수 종목"]
        quantity = input_data["매수 수량"]
        purchase_date = input_data["매수 일자"].replace("-", "")

        filtered_data = NHTI_df[(NHTI_df['tck_iem_cd'] == stock_code) & (NHTI_df['bse_dt'] == int(purchase_date))]

        if not filtered_data.empty:
            weight = (quantity / total_quantity)
            filtered_row = filtered_data.iloc[0]

            for score in weighted_scores:
                weighted_scores[score] += filtered_row[score] * weight

    def determine_nhti_type(risk, crowd, interest, volatility):
        return f"{'A' if risk >= 50 else 'B'}{'G' if crowd >= 50 else 'I'}{'P' if interest >= 50 else 'U'}{'X' if volatility >= 50 else 'S'}"

    nhti_type = determine_nhti_type(
        weighted_scores["Risk_Score"],
        weighted_scores["Crowd_Psychology_Score"],
        weighted_scores["Interest_Score"],
        weighted_scores["Volatility_Score"]
    )

    NHTI_df['Distance'] = np.sqrt(
        sum((NHTI_df[score] - weighted_scores[score])**2 for score in weighted_scores)
    )
    top_5_recommendations = NHTI_df.nsmallest(5, 'Distance')

    return nhti_type, weighted_scores, top_5_recommendations


## 파트 3

In [7]:
# 파트 3

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
import torch

# 모델 초기화 및 설정
model_name = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": "cuda"}  # 혹은 "cpu"로 변경해 시도
)

# 모델 파이프라인 설정 (stop_sequences 제거, max_new_tokens로 조절)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length = 2000, truncation=True)
llm = HuggingFacePipeline(pipeline=pipe)

# 프롬프트 템플릿 정의
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="""
=== NHTI 성향 분석 및 추천 종목 설명 ===

당신은 사용자의 투자 성향을 분석하는 전문가입니다. 사용자 투자 성향 점수와 추천 종목을 기반으로 각 성향 항목을 분석하고, 사용자에게 최적의 종목을 추천하세요. 간결하고 반복을 피하면서도 주요 성향을 연결해 설명해 주세요.

사용자 투자 성향 점수:
{question}

추천 종목 정보:
{context}

[분석 형식]
1. **Risk**: 사용자의 리스크 성향을 바탕으로 투자 경향을 설명하세요.
2. **Crowd Psychology**: 군중 심리 의존성에 대해 설명하고, 이 성향이 추천 종목에 어떻게 반영되는지 설명하세요.
3. **Interest**: 대중적으로 인기 있는 종목에 대한 사용자 성향을 설명하고, 추천 종목과 연결해 주세요.
4. **Volatility**: 변동성에 대한 관심을 간략히 설명하고, 이 성향에 맞는 종목을 제시하세요.

**결론**:
- 사용자 투자 성향을 한 문장으로 요약하고, 최적의 추천 종목 한두 개를 간결하게 제시하세요.
- 최적의 추천 종목: 선택된 종목과 사용자 성향 간의 연결성을 설명하고, 해당 종목들이 사용자의 투자 목표에 어떻게 부합하는지 요약하세요.
"""
)



tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

## 파트 4

In [8]:
# 파트 4

import pandas as pd
import numpy as np

# CSV 파일 불러오기
NHTI_df = pd.read_csv('/content/drive/MyDrive/NH 데이콘/본선/slopes_about_log_standardize_with_nhti.csv')

def get_user_inputs():
    user_inputs = []
    while True:
        stock_code = input("매수 종목을 입력하세요 (종료하려면 'q' 입력): ")
        if stock_code.lower() == 'q':
            break
        try:
            quantity = int(input("매수 수량을 입력하세요: "))
            purchase_date = input("매수 일자를 입력하세요 (YYYY-MM-DD 형식): ").replace("-", "")
            user_inputs.append({"매수 종목": stock_code, "매수 수량": quantity, "매수 일자": purchase_date})
        except ValueError:
            print("잘못된 입력입니다. 다시 시도하세요.")
    return user_inputs

def calculate_weighted_scores(user_inputs, total_quantity):
    weighted_scores = { "Risk_Score": 0, "Crowd_Psychology_Score": 0, "Interest_Score": 0, "Volatility_Score": 0 }
    for input_data in user_inputs:
        stock_code = input_data["매수 종목"]
        quantity = input_data["매수 수량"]
        purchase_date = input_data["매수 일자"]

        filtered_data = NHTI_df[(NHTI_df['tck_iem_cd'] == stock_code) & (NHTI_df['bse_dt'] == int(purchase_date))]

        if not filtered_data.empty:
            weight = (quantity / total_quantity) * 100
            filtered_row = filtered_data.iloc[0]
            for score in weighted_scores:
                weighted_scores[score] += filtered_row[score] * (weight / 100)
    weighted_scores = {k: round(v, 2) for k, v in weighted_scores.items()}
    return weighted_scores

def determine_nhti_type(weighted_scores):
    return (
        ('A' if weighted_scores["Risk_Score"] >= 50 else 'B') +
        ('G' if weighted_scores["Crowd_Psychology_Score"] >= 50 else 'I') +
        ('P' if weighted_scores["Interest_Score"] >= 50 else 'U') +
        ('X' if weighted_scores["Volatility_Score"] >= 50 else 'S')
    )

def recommend_stocks(weighted_scores):
    NHTI_df['Distance'] = np.sqrt(
        sum((NHTI_df[score] - weighted_scores[score])**2 for score in weighted_scores)
    )
    return NHTI_df.nsmallest(5, 'Distance')

# 전체 실행 함수
def calculate_nhti_with_input():
    user_inputs = get_user_inputs()
    total_quantity = sum(input_data["매수 수량"] for input_data in user_inputs)
    weighted_scores = calculate_weighted_scores(user_inputs, total_quantity)
    nhti_type = determine_nhti_type(weighted_scores)
    top_5_recommendations = recommend_stocks(weighted_scores)

    return nhti_type, weighted_scores, top_5_recommendations


## 파트 5

In [9]:
# 파트 5

def generate_nhti_analysis(nhti_type, weighted_scores, top_5_recommendations):
    question = f"""
    Risk Score: {weighted_scores['Risk_Score']:.2f}, Crowd Psychology Score: {weighted_scores['Crowd_Psychology_Score']:.2f},
    Interest Score: {weighted_scores['Interest_Score']:.2f}, Volatility Score: {weighted_scores['Volatility_Score']:.2f},
    NHTI 유형: {nhti_type}
    """

    context = "\n".join([

        f"- 종목: {row['tck_iem_cd']}, 날짜: {row['bse_dt']}, Risk Score: {row['Risk_Score']}, "
        f"Crowd Psychology Score: {row['Crowd_Psychology_Score']}, Interest Score: {row['Interest_Score']}, "
        f"Volatility Score: {row['Volatility_Score']}, 거리: {row['Distance']:.2f}"
        for _, row in top_5_recommendations.iterrows()
    ])

    prompt = prompt_template.format(question=question, context=context)

    try:
        response = llm(prompt, max_new_tokens=250)  # 충분한 텍스트 생성 설정
    except Exception as e:
        response = f"응답 생성에 실패했습니다: {e}"

    print("\n=== NHTI 성향 분석 및 추천 종목 설명 ===")
    print(response)


## 파트 6

In [10]:
# 파트 6

try:
    nhti_type, weighted_scores, top_5_recommendations = calculate_nhti_with_input()
    print(f"NHTI 유형: {nhti_type}")
    print("가중합 점수:", weighted_scores)
except Exception as e:
    print(f"NHTI 계산에 실패했습니다: {e}")
    top_5_recommendations = None

if top_5_recommendations is not None:
    try:
        generate_nhti_analysis(nhti_type, weighted_scores, top_5_recommendations)
    except Exception as e:
        print(f"NHTI 분석 및 추천 설명 생성에 실패했습니다: {e}")
else:
    print("추천 종목을 생성할 수 없습니다.")


매수 종목을 입력하세요 (종료하려면 'q' 입력): A
매수 수량을 입력하세요: 30
매수 일자를 입력하세요 (YYYY-MM-DD 형식): 2024-05-28
매수 종목을 입력하세요 (종료하려면 'q' 입력): A
매수 수량을 입력하세요: 50
매수 일자를 입력하세요 (YYYY-MM-DD 형식): 2024-05-31
매수 종목을 입력하세요 (종료하려면 'q' 입력): A
매수 수량을 입력하세요: 20
매수 일자를 입력하세요 (YYYY-MM-DD 형식): 2024-06-03
매수 종목을 입력하세요 (종료하려면 'q' 입력): q
NHTI 유형: AIUS
가중합 점수: {'Risk_Score': 79.6, 'Crowd_Psychology_Score': 29.6, 'Interest_Score': 30.4, 'Volatility_Score': 0.0}



=== NHTI 성향 분석 및 추천 종목 설명 ===

=== NHTI 성향 분석 및 추천 종목 설명 ===

당신은 사용자의 투자 성향을 분석하는 전문가입니다. 사용자 투자 성향 점수와 추천 종목을 기반으로 각 성향 항목을 분석하고, 사용자에게 최적의 종목을 추천하세요. 간결하고 반복을 피하면서도 주요 성향을 연결해 설명해 주세요.

사용자 투자 성향 점수:

    Risk Score: 79.60, Crowd Psychology Score: 29.60,
    Interest Score: 30.40, Volatility Score: 0.00,
    NHTI 유형: AIUS
    

추천 종목 정보:
- 종목: SOXL, 날짜: 20240802, Risk Score: 71, Crowd Psychology Score: 15, Interest Score: 29, Volatility Score: 5, 거리: 17.72
- 종목: SOXX, 날짜: 20240805, Risk Score: 75, Crowd Psychology Score: 14, Interest Score: 21, Volatility Score: 0, 거리: 18.79
- 종목: SOXL, 날짜: 20240805, Risk Score: 71, Crowd Psychology Score: 21, Interest Score: 45, Volatility Score: 6, 거리: 19.93
- 종목: POWL, 날짜: 20240731, Risk Score: 85, Crowd Psychology Score: 28, Interest Score: 11, Volatility Score: 0, 거리: 20.20
- 종목: TELL, 날짜: 20240722, Risk Score: 86, Crowd Psychology Score: 16, Interest Score: 17, Volatility Score: 2, 거리: 20.24

[분석 형식]
1. **Risk**: 사용자의 리스크 성향을 바탕으로 투자 경향을 설명하세

# ver_3

In [4]:
import os
import re
from dotenv import load_dotenv
import pandas as pd
import chromadb
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
import tqdm

# CSV 파일 불러오기
NHTI_df = pd.read_csv('/content/drive/MyDrive/NH 데이콘/본선/slopes_about_log_standardize_with_nhti.csv')

# 임베딩 모델 초기화
emb_model = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# Chroma DB에 문서 형식으로 데이터 저장
documents = []
feature_columns = [col for col in NHTI_df.columns if col not in ['tck_iem_cd', 'bse_dt']]

for row in tqdm.tqdm(NHTI_df[:20000].itertuples()):
    metadata = {
        "date": row.bse_dt,
        "stock": row.tck_iem_cd,
        **{feature: str(getattr(row, feature)) for feature in feature_columns}
    }
    query = f"Stock: {row.tck_iem_cd} Date: {row.bse_dt} " + " ".join([f"{feature}: {getattr(row, feature)}" for feature in feature_columns])
    doc = Document(page_content=query, metadata=metadata)
    documents.append(doc)

# Chroma DB 설정 및 데이터 저장
DB_PATH = "/content/chroma_db"
docsearch = Chroma.from_documents(documents, emb_model,
                                  persist_directory=DB_PATH,
                                  collection_name="stocks")


config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

RuntimeError: Expected one of cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, fpga, maia, xla, lazy, vulkan, mps, meta, hpu, mtia, privateuseone device type at start of device string: gpu

In [ ]:
import numpy as np

def calculate_nhti_with_input():
    user_inputs = []

    # 사용자로부터 매수 종목, 매수 수량, 매수 일자 입력 받기
    while True:
        stock_code = input("매수 종목을 입력하세요 (종료하려면 'q' 입력): ")
        if stock_code.lower() == 'q':
            break
        try:
            quantity = int(input("매수 수량을 입력하세요: "))
            purchase_date = input("매수 일자를 입력하세요 (YYYY-MM-DD 형식): ")
            user_inputs.append({"매수 종목": stock_code, "매수 수량": quantity, "매수 일자": purchase_date})
        except ValueError:
            print("잘못된 입력입니다. 다시 시도하세요.")

    total_quantity = sum(input_data["매수 수량"] for input_data in user_inputs)
    weighted_scores = {key: 0 for key in ["Risk_Score", "Crowd_Psychology_Score", "Interest_Score", "Volatility_Score"]}

    for input_data in user_inputs:
        stock_code = input_data["매수 종목"]
        quantity = input_data["매수 수량"]
        purchase_date = input_data["매수 일자"].replace("-", "")

        filtered_data = NHTI_df[(NHTI_df['tck_iem_cd'] == stock_code) & (NHTI_df['bse_dt'] == int(purchase_date))]

        if not filtered_data.empty:
            weight = (quantity / total_quantity)
            filtered_row = filtered_data.iloc[0]

            for score in weighted_scores:
                weighted_scores[score] += filtered_row[score] * weight

    def determine_nhti_type(risk, crowd, interest, volatility):
        return f"{'A' if risk >= 50 else 'B'}{'G' if crowd >= 50 else 'I'}{'P' if interest >= 50 else 'U'}{'X' if volatility >= 50 else 'S'}"

    nhti_type = determine_nhti_type(
        weighted_scores["Risk_Score"],
        weighted_scores["Crowd_Psychology_Score"],
        weighted_scores["Interest_Score"],
        weighted_scores["Volatility_Score"]
    )

    NHTI_df['Distance'] = np.sqrt(
        sum((NHTI_df[score] - weighted_scores[score])**2 for score in weighted_scores)
    )
    top_5_recommendations = NHTI_df.nsmallest(5, 'Distance')

    return nhti_type, weighted_scores, top_5_recommendations


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
import torch

# 모델 초기화 및 설정
model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": "cuda"}
)

# 모델 파이프라인 설정
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=2000, truncation=True)
llm = HuggingFacePipeline(pipeline=pipe)

# 영어로 프롬프트 템플릿 정의
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="""
=== NHTI Personality Analysis and Stock Recommendations ===

You are an expert in analyzing investment personalities. Based on the user's investment scores and recommended stocks, analyze each personality aspect and recommend optimal stocks. Explain concisely and avoid repetition, linking the key traits together.

User's Investment Personality Scores:
{question}

Recommended Stock Information:
{context}

[Analysis Format]
1. **Risk**: Describe the user's risk-taking tendency.
2. **Crowd Psychology**: Explain the dependency on crowd psychology and how this reflects in the recommended stocks.
3. **Interest**: Describe the user's interest in popular stocks, linking this to the recommended stocks.
4. **Volatility**: Briefly describe the focus on volatility and recommend stocks that match this tendency.

**Conclusion**:
- Summarize the user's investment personality in one sentence and suggest one or two optimal stocks.
- **Top Recommendation**: Describe how the selected stocks align with the user's investment goals.
"""
)


In [ ]:
def generate_nhti_analysis(nhti_type, weighted_scores, top_5_recommendations):
    question = f"""
    Risk Score: {weighted_scores['Risk_Score']:.2f}, Crowd Psychology Score: {weighted_scores['Crowd_Psychology_Score']:.2f},
    Interest Score: {weighted_scores['Interest_Score']:.2f}, Volatility Score: {weighted_scores['Volatility_Score']:.2f},
    NHTI Type: {nhti_type}
    """

    context = "\n".join([
        f"- Stock: {row['tck_iem_cd']}, Date: {row['bse_dt']}, Risk Score: {row['Risk_Score']}, "
        f"Crowd Psychology Score: {row['Crowd_Psychology_Score']}, Interest Score: {row['Interest_Score']}, "
        f"Volatility Score: {row['Volatility_Score']}, Distance: {row['Distance']:.2f}"
        for _, row in top_5_recommendations.iterrows()
    ])

    prompt = prompt_template.format(question=question, context=context)

    try:
        response = llm(prompt, max_new_tokens=250)
    except Exception as e:
        response = f"Failed to generate response: {e}"

    print("\n=== NHTI Personality Analysis and Recommendations ===")
    print(response)


In [ ]:
try:
    nhti_type, weighted_scores, top_5_recommendations = calculate_nhti_with_input()
    print(f"NHTI Type: {nhti_type}")
    print("Weighted Scores:", weighted_scores)
except Exception as e:
    print(f"Failed to calculate NHTI: {e}")
    top_5_recommendations = None

if top_5_recommendations is not None:
    try:
        generate_nhti_analysis(nhti_type, weighted_scores, top_5_recommendations)
    except Exception as e:
        print(f"Failed to generate NHTI analysis and recommendations: {e}")
else:
    print("Unable to generate stock recommendations.")
